### Basic ML Model Monitoring with MLFlow
**Description**: Set up a basic ML model monitoring pipeline using MLFlow to track experiment parameters and results.

**Steps**:
1. Installation
2. Set Up MLFlow Tracking
3. Training a Simple Model
4. Logging Model and Metrics
5. View Logged Data
    - Access the MLFlow UI to view your logged parameters, metrics, and models.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn
import os
import logging
import unittest

logging.basicConfig(level=logging.INFO)

# -------------------------------
# Step 1: Data Generation
# -------------------------------
def generate_data(n=1000):
    """
    Generate synthetic binary classification dataset for monitoring.
    """
    if n <= 0:
        raise ValueError("Data size must be greater than 0")

    data = pd.DataFrame({
        "feature1": np.random.normal(50, 10, n),
        "feature2": np.random.uniform(0, 1, n),
        "feature3": np.random.poisson(2, n),
        "target": np.random.choice([0, 1], n, p=[0.6, 0.4])
    })
    return data

# -------------------------------
# Step 2: Training & Evaluation
# -------------------------------
def train_model(data):
    """
    Train Logistic Regression model with standard scaling.
    """
    try:
        if data.empty or "target" not in data.columns:
            raise ValueError("Invalid or empty dataset. Check columns and size.")

        X = data.drop("target", axis=1)
        y = data["target"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = LogisticRegression()
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred)
        }

        return model, scaler, metrics

    except Exception as e:
        logging.error(f"Training failed: {e}")
        raise

# -------------------------------
# Step 3: MLflow Logging
# -------------------------------
def log_to_mlflow(model, scaler, metrics):
    """
    Log model, scaler, and metrics to MLflow.
    """
    try:
        mlflow.set_experiment("Basic_ML_Monitoring")

        with mlflow.start_run():
            mlflow.log_param("model", "LogisticRegression")
            mlflow.log_param("scaler", "StandardScaler")
            for k, v in metrics.items():
                mlflow.log_metric(k, v)

            mlflow.sklearn.log_model(model, artifact_path="model")
            mlflow.sklearn.log_model(scaler, artifact_path="scaler")

            logging.info("Logged to MLflow successfully.")

    except Exception as e:
        logging.error(f"MLflow logging failed: {e}")
        raise

# -------------------------------
# Step 4: Unit Tests
# -------------------------------
class TestPipeline(unittest.TestCase):

    def test_data_generation(self):
        df = generate_data(100)
        self.assertEqual(df.shape[0], 100)
        self.assertIn("target", df.columns)

    def test_training(self):
        df = generate_data(200)
        model, scaler, metrics = train_model(df)
        self.assertIn("accuracy", metrics)
        self.assertGreater(metrics["accuracy"], 0)

    def test_training_with_invalid_data(self):
        with self.assertRaises(ValueError):
            train_model(pd.DataFrame())

# -------------------------------
# Step 5: Main Entry Point
# -------------------------------
def main():
    print("🚀 ML Monitoring with MLflow\n")

    try:
        df = generate_data(1000)
        model, scaler, metrics = train_model(df)
        log_to_mlflow(model, scaler, metrics)
        print("✅ Monitoring pipeline complete. Run `mlflow ui` to view results.")
    except Exception as e:
        print(f"❌ Pipeline failed: {e}")

if __name__ == "__main__":
    main()

    print("\n🔍 Running unit tests...\n")
    unittest.main(argv=[''], exit=False)

ModuleNotFoundError: No module named 'mlflow'